<a href="https://colab.research.google.com/github/blackwithwhitegreen/kaggle-template/blob/main/Seasonality_Factors_Analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
suvroo_inventory_optimization_for_retail_path = kagglehub.dataset_download('suvroo/inventory-optimization-for-retail')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df1 = pd.read_csv('/kaggle/input/inventory-optimization-for-retail/demand_forecasting.csv').fillna(0)
df1

In [ ]:
df1.info()

In [ ]:
df1.columns

In [ ]:
# COUNT PLOT FOR CATEGORICAL COLUMNS
cate_cols =['Promotions', 'Seasonality Factors', 'External Factors', 'Demand Trend',
       'Customer Segments']
for features in cate_cols:
    if features in df1.columns:
        plt.figure(figsize=(10,5))
        sns.countplot(x=features,data=df1,palette='viridis')
        plt.title(f'Count plot of {features}')
        plt.tight_layout()
        plt.show()

#PLOT HISTOGRAM FOR SEVERAL KEY NUM_FEATURE.
num_feature = ['Promotions', 'Seasonality Factors', 'External Factors', 'Demand Trend',
       'Customer Segments']

for feature in num_feature:
    if feature in df1.columns:
        plt.figure(figsize=(10,5))
        sns.histplot(df1[feature].dropna(),kde=True,color='purple')
        plt.title(f'Histogram of {feature}')
        plt.xlabel(feature)
        plt.ylabel('Count')
        plt.tight_layout()
        plt.show()

nume_df = df1.select_dtypes(include=[np.number])
if nume_df.shape[1]>=4:
    plt.figure(figsize=(10,5))
    correlation_matrix = nume_df.corr()
    sns.heatmap(correlation_matrix,annot=True,cmap='coolwarm',fmt='.2f')
    plt.title('Correlation matrix of Numeric Features')
    plt.tight_layout()
    plt.show()

# PAIR-PLOT AMONG A SUBSET OF VARIABLES
selected_columns = ['Promotions', 'Seasonality Factors', 'External Factors', 'Demand Trend',
       'Customer Segments']
if set(selected_columns).issubset(df1.columns):
    sns.pairplot(df1[selected_columns].dropna(),diag_kind='kde')
    plt.show()

# DATA PREPROCESSING
* Here just dividing the 0 equally in both the values same with the external feaurue column alos.

In [ ]:
# Count occurrences of each category
total_val = df1['Seasonality Factors'].value_counts()

# Get counts for Holiday, Festival, and Zero
Holiday_count = total_val.get('Holiday', 0)
Festival_count = total_val.get('Festival', 0)
zero_count = total_val.get(0, 0)

# If there are no 'Zero' values, no need to proceed further
if zero_count > 0:

    # Distribute Zero equally
    zero_to_Holiday = zero_count // 2
    zero_to_Festival = zero_count - zero_to_Holiday

    # Get indices of rows where 'Seasonality Factors' is 'Zero'
    zero_indices = df1[df1['Seasonality Factors'] == 0].index

    # Update first half to 'Holiday' and second half to 'Festival'
    df1.loc[zero_indices[:zero_to_Festival], 'Seasonality Factors'] = 'Holiday'
    df1.loc[zero_indices[zero_to_Holiday:], 'Seasonality Factors'] = 'Festival'

# Print updated value counts
print(df1['Seasonality Factors'].value_counts())

In [ ]:
df1.info()

In [ ]:
df1['External Factors'].value_counts()

In [ ]:
# Count occurrences of each category
total_val = df1['External Factors'].value_counts()

# Get counts for Holiday, Festival, and Zero
Weather_count = total_val.get('Weather', 0)
Competitor_pricing_count = total_val.get('Competitor Pricing', 0)
Economic_indicator_count = total_val.get('Economic Indicator',0)
zero_count = total_val.get(0, 0)

# If there are no 'Zero' values, no need to proceed further
if zero_count > 0:

    # Distribute Zero equally

    zero_to_weather = zero_count // 3
    zero_to_competitor  = zero_count // 3
    zero_to_economic = zero_count - (zero_to_weather + zero_to_competitor)

    # Get indices of rows where 'Seasonality Factors' is 'Zero'
    zero_indices = df1[df1['External Factors'] == 0].index

    # Update first half to 'Holiday' and second half to 'Festival'
    df1.loc[zero_indices[:zero_to_weather], 'External Factors'] = 'Weather'

    df1.loc[zero_indices[zero_to_weather : zero_to_weather + zero_to_competitor],'External Factors'] = 'Competitor Pricing'

    df1.loc[zero_indices[zero_to_weather + zero_to_competitor:], 'External Factors'] = 'Economic Indicator'

# Print updated value counts
print(df1['External Factors'].value_counts())

In [ ]:
df1.info()

In [ ]:
df1['Seasonality Factors']

In [ ]:
df1 = df1.drop(columns=['Product ID', 'Date', 'Store ID', 'Sales Quantity', 'Price'])

In [ ]:
df1.info()

In [ ]:
df1.columns

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()

In [ ]:
le_cols = ['Promotions', 'Seasonality Factors', 'External Factors', 'Demand Trend',
       'Customer Segments']
for column in le_cols:
    df1[column] = le.fit_transform(df1[column])

In [ ]:
df1.columns

# MODEL SECTION

In [ ]:
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error,r2_score,accuracy_score,mean_absolute_error
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
X = df1.drop(columns=['Seasonality Factors'])
y = df1['Seasonality Factors']

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42,test_size=0.2)

In [ ]:
dtr = DecisionTreeRegressor()

In [ ]:
dtr.fit(X_train,y_train)

In [ ]:
y_pred_dtr = dtr.predict(X_test)

In [ ]:
mse = mean_squared_error(y_test,y_pred_dtr)
r2score = r2_score(y_test,y_pred_dtr)
mae = mean_absolute_error(y_test,y_pred_dtr)
print(f'MSE:',mse)
print(f'R2-Score:',r2score)
print(f'MAE:',mae)

In [ ]:
# Define the parameter grid to tune the hyperparameters
param_grid = {
    'max_depth': [10, 20, 30,40,50,60, None],
    'min_samples_split': [2, 5, 10,15,20,25,30],
    'min_samples_leaf': [1, 2, 4,6,8,10,15,20]
}
dtree_reg = DecisionTreeClassifier(random_state=42) # Initialize a decision tree regressor
grid_search = GridSearchCV(estimator=dtree_reg, param_grid=param_grid,
                           cv=5, n_jobs=-1, verbose=5, scoring='r2')
grid_search.fit(X_train, y_train)
best_dtree_reg = grid_search.best_estimator_ # Get the best estimator from the grid search
y_pred = best_dtree_reg.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test,y_pred)
r2_scr = r2_score(y_test,y_pred)
rmse = mse ** 0.5
rmae = mae ** 0.5
best_params = grid_search.best_params_
print(f"Best parameters: {best_params}")
print(f"Test RMSE: {rmse}")
print(f"Test RMAE:' {rmae}")
print(f"Test R2 Score:{r2_scr}")

In [ ]:
from sklearn.svm import SVC

In [ ]:
sv = SVC()

In [ ]:
sv.fit(X_train,y_train)

In [ ]:
y_pred_sv = sv.predict(X_test)

In [ ]:
acc_sv = accuracy_score(y_pred_sv,y_test)
print(f'acc_sv:',acc_sv*100)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
rfr = RandomForestRegressor()

In [ ]:
rfr.fit(X_train,y_train)

In [ ]:
y_pred_rfr = rfr.predict(X_test)

In [ ]:
mse = mean_squared_error(y_test,y_pred_rfr)
r2score = r2_score(y_test,y_pred_rfr)
mae = mean_absolute_error(y_test,y_pred_rfr)
print(f'MSE:',mse)
print(f'R2-Score:',r2score)
print(f'MAE:',mae)

In [ ]:
rfc = RandomForestClassifier()

In [ ]:
rfc.fit(X_train,y_train)

In [ ]:
y_pred_rfc = rfc.predict(X_test)

In [ ]:
acc_rfc = accuracy_score(y_pred_sv,y_test)
print(f'acc_rfc:',acc_rfc*100)

In [ ]:
gb = GradientBoostingClassifier()

In [ ]:
gb.fit(X_train,y_train)

In [ ]:
y_pred_gb = gb.predict(X_test)

In [ ]:
acc_gb = accuracy_score(y_pred_sv,y_test)
print(f'acc_rfc:',acc_gb*100)

In [ ]:
kn = KNeighborsClassifier()

In [ ]:
kn.fit(X_train,y_train)

In [ ]:
y_pred_knn = kn.predict(X_test)

In [ ]:
acc_kn = accuracy_score(y_pred_knn,y_test)
print(f'acc_rfc:',acc_kn*100)

# CONCLUSION :
> Need to imporve the Accuracy socre, and r2 score.